# Code to create training dataset for model based on the data collected from the camera.
(usefule link: https://saturncloud.io/blog/how-to-write-text-in-jupyter-ipython-notebook/)

In [3]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

DATA_DIR = './data'
TRANSLATED_DATA_DIR = './data_translated'

In [4]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

In [5]:
data, labels = [], []
for dir_ in os.listdir(DATA_DIR):
    path = os.path.join(DATA_DIR, dir_)
    translated_data_path = os.path.join(TRANSLATED_DATA_DIR, dir_)
    if not os.path.exists(translated_data_path): os.makedirs(translated_data_path)
        
    for img_path in os.listdir(path):
        data_aux, x_, y_ = [], [], []

        img = cv2.imread(os.path.join(path, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,  
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                cv2.imwrite(os.path.join(translated_data_path, img_path), img)
                
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
    
                    x_.append(x)
                    y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        data.append(data_aux)
        labels.append(dir_)

In [6]:
f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()